In [1]:
"""
We want to take a look at using the AST library.
Below is a simple function that we will be examining using ASTs.
"""
import inspect

def calculate_average(numbers):
    total = 0
    for element in numbers:
        total += element
    average = total / len(numbers)
    print(average)
    return average

code = inspect.getsource(calculate_average)
print("=== code ===")
print(code)

=== code ===
def calculate_average(numbers):
    total = 0
    for element in numbers:
        total += element
    average = total / len(numbers)
    print(average)
    return average



As far as using the AST goes, some of the basic features we would want to satisfy are: 
* To be able to pass the source code above into an AST and iterate through it, node by node
* At each node, to be able to get some meaning out of the type of node we're looking at (e.g. does this node represent a 'for loop'?)
* Supposing we had a list of rules similar to those we created for structural explanations. Could we check to see that the rules are met in order as we proceed through the ast walk?

As an end goal, try to write a program that checks for the existence of a 'for loop', a 'print statement', and a 'return statement', in that order.

Some resources to get you started:
(but by no means limit yourself to these; look for more, for ones that you understand best)
* https://www.youtube.com/watch?v=OjPT15y2EpE
* https://medium.com/@wshanshan/intro-to-python-ast-module-bbd22cd505f7
* https://earthly.dev/blog/python-ast/
* https://docs.python.org/3/library/ast.html

In [ ]:
import ast
import pprint

tree = ast.parse(code)
# pprint.pprint(ast.dump(tree))

print("=== ast walk ===")

"""
Here we walk through the ast, going 1 node at a time.
The nodes do not correspond one-to-one with each line of code, but they are still linked in meaning.
Try to read through the printout of the walk and see how each node in the walk corresponds to an element of the given code.
(In particular, the 'lineno' or 'line number' field in the printout should be helpful for understanding the connection.)
"""
for node in ast.walk(tree):
    print(node)
    print(node.__dict__)
    # print(node._fields)
    print("children: " + str([x for x in ast.iter_child_nodes(node)]) + "\n")

    # How could we check to see if it is a certain type of node?
    # ...

    # If we were to go further than that, suppose we had a list of rules similar to those we created for structural explanations...
    # Could we check to see that the rules are met in order as we proceed through the ast walk?
    # ...

In [70]:
def first_exploration(code):
    """
    === code === (Reminder)
    def calculate_average(numbers):
        total = 0
        for element in numbers:
            total += element
        average = total / len(numbers)
        print(average)
        return average
    """
    node = ast.parse(code)
    print(node)
    print(node._fields)
    print()
    
    print(node.body)
    function_body_node = node.body[0]
    print(function_body_node)
    print(function_body_node._fields)
    print(function_body_node.body)  # see how this corresponds to a broad, breadth-type stroke across the AST?
    print()

    for_node = function_body_node.body[1]
    print(for_node)
    print(for_node._fields)
    print(for_node.body)  # and see how we are now inside the for loop?

    aug_assign_in_for_loop_node = for_node.body[0]
    print(aug_assign_in_for_loop_node)
    print(aug_assign_in_for_loop_node._fields)
    print(aug_assign_in_for_loop_node.target)  # and see now how can't go any deeper? 
    print(aug_assign_in_for_loop_node.op)
    print(aug_assign_in_for_loop_node.value)
    print()

    # so if we were to summarize this entire exploration here, we can see the path we took:
    # function definition -> entire function body -> just the for loop -> body of the for loop -> the aug assign node inside it.
    
    # can we check the type of a node here?
    # let's try:
    if isinstance(for_node, ast.For):
        print("yes, this works to check type.")
        print()

    # great, so could check structure based on that.

    # so if we were to look for a set of rules based on this, how would we do it?
    # store a list of (ast.__) types that we want to check against
    # do an ast walk through the list, matching each type in the list against something...
    # but wait! if we start the walk from the root, we will find more than we are really after. We'll be searching the entire context.

    # so instead, we first find the local context that we're looking for...
    # then we do check the rest.
    # do our ast walk through that local context...
    # will this work? Let's see:

    # for n in ast.walk(for_node): 
    #     print(n)
    #     # print(node.__dict__)
    #     # print("children: " + str([x for x in ast.iter_child_nodes(node)]) + "\n")
    # print()
    
    # for n in ast.walk(aug_assign_in_for_loop_node):
    #     print(n)
    
first_exploration(code)

('body', 'type_ignores')

('name', 'args', 'body', 'decorator_list', 'returns', 'type_comment')
[<ast.Assign object at 0x0000026AC1D27C40>, <ast.For object at 0x0000026AC1D25BD0>, <ast.Assign object at 0x0000026AC1D26F80>, <ast.Expr object at 0x0000026AC1D25B10>, <ast.Return object at 0x0000026AC1D278E0>]

('target', 'iter', 'body', 'orelse', 'type_comment')
('target', 'op', 'value')

yes, this works to check type.



In [72]:
def attempt(code):
    """
    In which we will try to confirm the following structural ruleset:
    [for, increment]
    i.e., that the code has a for loop wherein something is incremented
    """
    ruleset = [ast.For, ast.Add]
    root = ast.parse(code)
    
    # here are all the contexts at the highest level inside the source code
    top_level_context_nodes = root.body[0].body
    print(top_level_context_nodes)

    # try to find the correct local context to match the first rule in the ruleset
    top_context_rule = ruleset[0]
    for top_node in top_level_context_nodes:
        print(top_node)
        
        if isinstance(top_node, top_context_rule):
            print(f"entering local context for rule at node {top_node}")
            return check_local_context(top_node, ruleset[1:])

def check_local_context(local_context_node, rules):
    rule_counter = 0
    for node in ast.walk(local_context_node):
        # if node matches structural requirement, then increment rule counter
        if isinstance(node, rules[rule_counter]):
            print(f"matching rule {rules[rule_counter]} on node {node}")
            rule_counter += 1
            if rule_counter >= len(rules):
                return "Structure fully matched!"
        # if not, do nothing

    return (f"Failed to match structure on rule {rules[rule_counter]}")

ans = attempt(code)
print(ans)

[<ast.Assign object at 0x0000026AC1D214B0>, <ast.For object at 0x0000026AC1D20190>, <ast.Assign object at 0x0000026AC1D212A0>, <ast.Expr object at 0x0000026AC1D20A30>, <ast.Return object at 0x0000026AC1D21420>]
entering local context for rule at node <ast.For object at 0x0000026AC1D20190>
matching rule <class 'ast.Add'> on node <ast.Add object at 0x0000026ABD91DC90>
Structure fully matched!


In [ ]:
"""
Some older code you might find useful as a reference.
This method uses the 'NodeVisitor' approach to move through the AST, different from the ast walk approach seen above.

These are magic functions, which are called depending on the name.
The 'visit_Call()' function is called on all nodes that are of 'call' type
The 'visit_For()' function is called on all nodes that are of 'for' type
"""
class FunctionCallVisitor(ast.NodeVisitor):
    def visit_Call(self, node):
        if isinstance(node.func, ast.Name) and node.func.id == "print":
            print(f"yup found a line with a print. It is this line: {node.__dict__}")
            args = [arg for arg in node.args if isinstance(arg, ast.Constant)]
            if args:
                print("Detected print statements with string literals:")
                for arg in args:
                    print(arg.s)  # Print the string literal directly
        self.generic_visit(node)

    def visit_For(self, node):
        # if isinstance(node.func, ast.Name) and node.func.id == "print":
        # print(f"args = {node.args}")
        # args = [arg for arg in node.args if isinstance(arg, ast.Constant)]
        # print(f"node.func = {node.func}")
        # args = [arg for arg in node.args if isinstance(arg, ast.For)]
        # if args:

        # class ast.For(target, iter, body, orelse, type_comment)
        # https://docs.python.org/3/library/ast.html > ctrl+f > ast.for

        print("Detected For statement with...")
        print(f"node.target={node.target} \nnode.iter={node.iter} \nnode.body={node.body} \nnode.orelse={node.orelse} \nnode.type_comment={node.type_comment}")

        # Look for initial loop construct node, 
        # then do a simple tree traversal starting at that loop node

def perform_static_analysis(code):
    """
    === code === (Reminder)
    def calculate_average(numbers):
        total = 0
        for element in numbers:
            total += element
        average = total / len(numbers)
        print(average)
        return average
    """
    tree = ast.parse(code)
    visitor = FunctionCallVisitor()
    visitor.visit(tree)

perform_static_analysis(code)